<a href="https://colab.research.google.com/github/mianyumifen-bot/codePublic/blob/main/%E5%90%8E%E7%BB%AD%E5%A4%84%E7%90%86NDWI%E6%96%87%E4%BB%B6%EF%BC%88%E5%A4%87%E4%BB%BD%EF%BC%8C%E8%BD%AC%E7%A7%BB%EF%BC%8C%E6%94%B9%E5%90%8D%EF%BC%89%EF%BC%882025_11_05%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
# ======================================================
#  第1单元格：环境设置与完整参数配置
# ======================================================

# 1. 安装与导入必要的库
!pip install --quiet earthengine-api geemap rasterio
import os
import re
import shutil
import datetime
import ee
import rasterio
from google.colab import drive

# 2. 授权GEE与挂载Google Drive
try:
    # 填入你自己的GEE项目名称
    ee.Initialize(project="ee-mianyumifen")
except Exception:
    ee.Authenticate()
    ee.Initialize(project="ee-mianyumifen")
print("Google Earth Engine 已成功初始化。")

drive.mount('/content/drive', force_remount=True)
print("谷歌硬盘已成功挂载。")

# 3. 【重要】核心配置参数
#    !!! 请确保这里的文件夹名称与您之前GEE导出脚本中的完全一致 !!!
DRIVE_ROOT = '/content/drive/MyDrive'

# 中转站文件夹：所有文件处理的起点
STAGING_EXPORT_FOLDER = 'S2_700_NDWI_Final_Exports'

# 备份文件夹：用于存放中转站里的“(副本)”文件
BACKUP_FOLDER_NAME = 'S2_700_NDWI_Final_Exports_备份'

# 最终输出文件夹：文件整理的目的地，也是修复脚本的目标
FINAL_OUTPUT_ROOT_FOLDER = 'S2_700_NDWI_Yearly_Composites_Final'

# 构建所有需要的完整路径
staging_path = os.path.join(DRIVE_ROOT, STAGING_EXPORT_FOLDER)
backup_path = os.path.join(DRIVE_ROOT, BACKUP_FOLDER_NAME)
final_path = os.path.join(DRIVE_ROOT, FINAL_OUTPUT_ROOT_FOLDER)

print(f"\n配置加载完成:")
print(f" - 中转站 (源): {staging_path}")
print(f" - 备份文件夹 (用于副本): {backup_path}")
print(f" - 最终目录 (目的地与修复目标): {final_path}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.8 MB/s eta 0:00:00
Google Earth Engine 已成功初始化。
Mounted at /content/drive
谷歌硬盘已成功挂载。

配置加载完成:
 - 中转站 (源): /content/drive/MyDrive/S2_700_NDWI_Final_Exports
 - 备份文件夹 (用于副本): /content/drive/MyDrive/S2_700_NDWI_Final_Exports_备份
 - 最终目录 (目的地与修复目标): /content/drive/MyDrive/S2_700_NDWI_Yearly_Composites_Final


In [2]:
# -*- coding: utf-8 -*-
# ======================================================
#  第2单元格：【步骤1】移动副本文件到备份文件夹
# ======================================================

print("开始检查并移动副本文件...")

# 确保备份文件夹存在
os.makedirs(backup_path, exist_ok=True)
moved_count = 0

if not os.path.exists(staging_path):
    print(f"警告：找不到中转文件夹 '{staging_path}'，跳过此步骤。")
else:
    # 遍历源文件夹中的所有文件
    for filename in os.listdir(staging_path):
        # 检查文件名是否包含中文的“（副本）”
        if '（副本）' in filename:
            try:
                source_file = os.path.join(staging_path, filename)
                destination_file = os.path.join(backup_path, filename)
                shutil.move(source_file, destination_file)
                print(f"  已备份副本文件: {filename}")
                moved_count += 1
            except Exception as e:
                print(f"  移动文件 {filename} 时出错: {e}")

    print("\n========================================================")
    if moved_count > 0:
        print(f"备份完成！共 {moved_count} 个副本文件已移动到备份文件夹。")
        print(f"现在的中转文件夹 '{STAGING_EXPORT_FOLDER}' 已经清理干净。")
    else:
        print("操作完成，在中转文件夹中没有找到需要备份的“(副本)”文件。")
    print("========================================================")

开始检查并移动副本文件...
  已备份副本文件: vancouversaltmarsh_2022.tif（副本）
  已备份副本文件: vancouversaltmarsh_2023.tif（副本）
  已备份副本文件: vancouversaltmarsh_2020.tif（副本）
  已备份副本文件: vancouversaltmarsh_2021.tif（副本）
  已备份副本文件: vancouversaltmarsh_2018.tif（副本）
  已备份副本文件: vancouversaltmarsh_2016.tif（副本）
  已备份副本文件: vancouversaltmarsh_2024.tif（副本）
  已备份副本文件: vancouversaltmarsh_2025.tif（副本）
  已备份副本文件: vancouversaltmarsh_2019.tif（副本）
  已备份副本文件: vancouversaltmarsh_2017.tif（副本）
  已备份副本文件: siwetland_2025.tif（副本）
  已备份副本文件: siwetland_2017.tif（副本）
  已备份副本文件: siwetland_2018.tif（副本）
  已备份副本文件: siwetland_2019.tif（副本）
  已备份副本文件: siwetland_2022.tif（副本）
  已备份副本文件: siwetland_2021.tif（副本）
  已备份副本文件: siwetland_2020.tif（副本）
  已备份副本文件: siwetland_2023.tif（副本）
  已备份副本文件: siwetland_2024.tif（副本）
  已备份副本文件: vancouversaltmarsh_2015.tif（副本）
  已备份副本文件: siwetland_2015.tif（副本）
  已备份副本文件: siwetland_2016.tif（副本）
  已备份副本文件: richmondbrackishmarsh_2021.tif（副本）
  已备份副本文件: richmondbrackishmarsh_2020.tif（副本）
  已备份副本文件: richmondbrackishmarsh_2018.tif（副本）


In [3]:
# -*- coding: utf-8 -*-
# ======================================================
#  第3单元格：【步骤2】整理文件 (从中转站到最终目录)
# ======================================================

print("\n开始整理剩余的有效文件...")

# 确保最终的目标根目录存在
os.makedirs(final_path, exist_ok=True)
moved_count = 0

if not os.path.exists(staging_path):
    print(f"错误：找不到中转文件夹 '{staging_path}'。")
else:
    for filename in os.listdir(staging_path):
        if filename.endswith('.tif'):
            try:
                site_name = filename.split('_')[0]
                site_folder = os.path.join(final_path, site_name)
                os.makedirs(site_folder, exist_ok=True)

                source_file = os.path.join(staging_path, filename)
                destination_file = os.path.join(site_folder, filename)
                shutil.move(source_file, destination_file)
                print(f"  已移动: {filename} -> {site_folder}/")
                moved_count += 1
            except Exception as e:
                print(f"  处理文件 {filename} 时出错: {e}")

    print("\n========================================================")
    print(f"整理完成！共移动了 {moved_count} 个有效文件到最终目录。")
    if moved_count > 0:
        print(f"提示：中转文件夹 '{STAGING_EXPORT_FOLDER}' 现已清空。")
    print("========================================================")


开始整理剩余的有效文件...
  已移动: cmarshhighsaltmarsh_2015.tif -> /content/drive/MyDrive/S2_700_NDWI_Yearly_Composites_Final/cmarshhighsaltmarsh/
  已移动: cmarshhighsaltmarsh_2016.tif -> /content/drive/MyDrive/S2_700_NDWI_Yearly_Composites_Final/cmarshhighsaltmarsh/
  已移动: cmarshhighsaltmarsh_2018.tif -> /content/drive/MyDrive/S2_700_NDWI_Yearly_Composites_Final/cmarshhighsaltmarsh/
  已移动: cmarshhighsaltmarsh_2017.tif -> /content/drive/MyDrive/S2_700_NDWI_Yearly_Composites_Final/cmarshhighsaltmarsh/
  已移动: cmarshhighsaltmarsh_2020.tif -> /content/drive/MyDrive/S2_700_NDWI_Yearly_Composites_Final/cmarshhighsaltmarsh/
  已移动: cmarshhighsaltmarsh_2019.tif -> /content/drive/MyDrive/S2_700_NDWI_Yearly_Composites_Final/cmarshhighsaltmarsh/
  已移动: cmarshhighsaltmarsh_2022.tif -> /content/drive/MyDrive/S2_700_NDWI_Yearly_Composites_Final/cmarshhighsaltmarsh/
  已移动: cmarshhighsaltmarsh_2021.tif -> /content/drive/MyDrive/S2_700_NDWI_Yearly_Composites_Final/cmarshhighsaltmarsh/
  已移动: cmarshhighsaltmarsh_2024.

In [8]:
# -*- coding: utf-8 -*-
# ======================================================
#  第4单元格：【步骤3】本地快速修复脚本 (全新修正版)
# ======================================================

print("\n开始运行本地快速修复脚本...")

if not os.path.exists(final_path):
    print(f"\n错误：找不到最终文件夹 '{final_path}'。")
else:
    repaired_count = 0
    skipped_count = 0
    error_count = 0

    # 遍历最终文件夹下的所有站点子文件夹
    for site_name in os.listdir(final_path):
        site_folder_path = os.path.join(final_path, site_name)
        if not os.path.isdir(site_folder_path): continue

        print(f"\n--- 正在处理站点: '{site_name}' ---")

        for filename in os.listdir(site_folder_path):
            if not filename.endswith('.tif'): continue

            file_path = os.path.join(site_folder_path, filename)
            print(f"  -> 正在检查文件: {filename}")

            try:
                with rasterio.open(file_path, 'r+') as dataset:
                    if not dataset.descriptions:
                        print("     文件无波段描述，跳过。"); continue

                    # 【关键修正】全新的判断逻辑：
                    # 正确的格式 'site_date_tile' 分割后是3段。
                    # 如果不是3段，就需要修复。
                    first_desc = dataset.descriptions[0]
                    if len(first_desc.split('_')) == 3:
                         print(f"     格式已正确 (例如: '{first_desc}'), 跳过。")
                         skipped_count += 1
                         continue

                    print(f"     检测到旧格式 (例如: '{first_desc}'), 开始修复...")
                    old_descriptions = dataset.descriptions
                    new_descriptions = []

                    for old_desc in old_descriptions:
                        # 从复杂的旧名称中精确提取需要的部分
                        # 提取日期: _YYYYMMDD_
                        date_match = re.search(r'_(\d{8})_', old_desc)
                        # 提取瓦片号: _T..._
                        tile_match = re.search(r'_(T\d{2}[A-Z]{3})_', old_desc)

                        if date_match and tile_match:
                            date_str = date_match.group(1)
                            tile_id = tile_match.group(1)

                            # 组合成新的、简单的名称: site_date_tile
                            new_desc = f"{site_name}_{date_str}_{tile_id}"
                            new_descriptions.append(new_desc)
                        else:
                            print(f"     警告: 无法从 '{old_desc}' 中提取信息，保留原样。")
                            new_descriptions.append(old_desc) # 保留旧名称

                    if len(new_descriptions) == len(old_descriptions):
                        dataset.descriptions = tuple(new_descriptions)
                        print(f"     [修复成功] {len(new_descriptions)} 个波段已重命名 (新格式例如: '{new_descriptions[0]}')")
                        repaired_count += 1
                    else:
                        print(f"     [修复失败] 新旧波段数量不匹配，已跳过。")
                        error_count += 1

            except Exception as e:
                print(f"     处理文件 {filename} 时发生严重错误: {e}")
                error_count += 1

    print("\n========================================================")
    print("所有修复操作完成！")
    print(f"共修复了 {repaired_count} 个文件。")
    print(f"共跳过了 {skipped_count} 个格式已正确的文件。")
    print(f"有 {error_count} 个文件在处理过程中出错。")
    print("========================================================")


开始运行本地快速修复脚本...

--- 正在处理站点: 'cmarshhighsaltmarsh' ---
  -> 正在检查文件: cmarshhighsaltmarsh_2015.tif
     格式已正确 (例如: 'cmarshhighsaltmarsh_20150809_T19TCH'), 跳过。
  -> 正在检查文件: cmarshhighsaltmarsh_2016.tif
     格式已正确 (例如: 'cmarshhighsaltmarsh_20160106_T19TCH'), 跳过。
  -> 正在检查文件: cmarshhighsaltmarsh_2018.tif
     格式已正确 (例如: 'cmarshhighsaltmarsh_20180103_T19TCH'), 跳过。
  -> 正在检查文件: cmarshhighsaltmarsh_2017.tif
     格式已正确 (例如: 'cmarshhighsaltmarsh_20170103_T19TCH'), 跳过。
  -> 正在检查文件: cmarshhighsaltmarsh_2020.tif
     格式已正确 (例如: 'cmarshhighsaltmarsh_20200103_T19TCH'), 跳过。
  -> 正在检查文件: cmarshhighsaltmarsh_2019.tif
     格式已正确 (例如: 'cmarshhighsaltmarsh_20190103_T19TCH'), 跳过。
  -> 正在检查文件: cmarshhighsaltmarsh_2022.tif
     格式已正确 (例如: 'cmarshhighsaltmarsh_20220102_T19TCH'), 跳过。
  -> 正在检查文件: cmarshhighsaltmarsh_2021.tif
     格式已正确 (例如: 'cmarshhighsaltmarsh_20210102_T19TCH'), 跳过。
  -> 正在检查文件: cmarshhighsaltmarsh_2024.tif
     格式已正确 (例如: 'cmarshhighsaltmarsh_20240102_T19TCH'), 跳过。
  -> 正在检查文件: cmarshhighsal

In [5]:
# -*- coding: utf-8 -*-
# ======================================================
#  第四单元格：外科手术式修复脚本 (针对性修复已知错误)
# ======================================================

import rasterio
import re
import os

print("开始运行针对性的“外科手术式”修复脚本...")

# --- 手动定义的修复地图 ---
# 在这里，我们明确指定哪个站点需要修复，以及它正确的瓦片ID是什么。
# 格式: { '站点文件夹名': '正确的瓦片ID' }
# 您可以根据需要添加更多条目。
manual_fix_map = {
    'norriepoint': 'T18TWM'
}

print("\n--- 将要执行的修复任务 ---")
print(manual_fix_map)
print("脚本将只处理以上列出的站点文件夹。")
# -----------------------------

repaired_count = 0
skipped_count = 0

if not os.path.exists(final_path):
    print(f"\n错误：找不到最终文件夹 '{final_path}'。")
else:
    # 遍历我们手动指定的需要修复的站点
    for site_name, correct_tile_id in manual_fix_map.items():
        site_folder_path = os.path.join(final_path, site_name)

        if not os.path.exists(site_folder_path):
            print(f"\n警告：在修复列表中指定的站点文件夹 '{site_name}' 不存在，已跳过。")
            continue

        print(f"\n--- 正在扫描目标站点: '{site_name}' (强制使用正确瓦片ID: {correct_tile_id}) ---")

        # 遍历目标站点文件夹中的所有TIF文件
        for filename in os.listdir(site_folder_path):
            if filename.endswith('.tif'):
                file_path = os.path.join(site_folder_path, filename)
                try:
                    print(f"  -> 正在检查文件: {filename}")
                    with rasterio.open(file_path, 'r+') as dataset:
                        if not dataset.descriptions:
                            print("     文件无波段描述，跳过。")
                            continue

                        first_desc = dataset.descriptions[0]

                        # 【关键】检查波段名是否是之前被损坏的格式 (T + 5个数字)
                        if re.search(r'T\d{5}', first_desc):
                            print(f"     [发现损伤!] 格式为: '{first_desc}'。正在应用修复...")
                            old_descriptions = dataset.descriptions
                            new_descriptions = []

                            for old_desc in old_descriptions:
                                # 从损坏的名称中提取仍然有用的日期和波段号
                                date_match = re.search(r'(\d{8})', old_desc)
                                date_str = date_match.group(1) if date_match else "NODATE"

                                band_match = re.search(r'(B\d{1,2}A?)', old_desc)
                                band_id = band_match.group(1) if band_match else "NOBAND"

                                # 使用我们手动指定的、100%正确的瓦片ID来组合新名称
                                new_desc = f"{date_str}_{band_id}_{correct_tile_id}"
                                new_descriptions.append(new_desc)

                            # 将修复后的名称写回文件
                            dataset.descriptions = tuple(new_descriptions)
                            print(f"     [修复成功] 已重命名为 (例如): '{new_descriptions[0]}'")
                            repaired_count += 1
                        else:
                            print(f"     格式正确或无需修复: '{first_desc}', 跳过。")
                            skipped_count += 1

                except Exception as e:
                    print(f"处理文件 {filename} 时出错: {e}")

print("\n========================================================")
print("修复完成！")
print(f"共修复了 {repaired_count} 个文件。")
print(f"共跳过了 {skipped_count} 个无需修复的文件。")
print("========================================================")

开始运行针对性的“外科手术式”修复脚本...

--- 将要执行的修复任务 ---
{'norriepoint': 'T18TWM'}
脚本将只处理以上列出的站点文件夹。

警告：在修复列表中指定的站点文件夹 'norriepoint' 不存在，已跳过。

修复完成！
共修复了 0 个文件。
共跳过了 0 个无需修复的文件。
